In [1]:
from bs4 import BeautifulSoup
import requests


# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]


def fetch_website_links(url):
    """
    Return the links on the webiste at the given url
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    links = [link.get("href") for link in soup.find_all("a")]
    return [link for link in links if link]

In [5]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [7]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [15]:
links = fetch_website_links("https://www.drikpanchang.com/")
links

['/',
 '/',
 'https://play.google.com/store/apps/details?id=com.drikp.core&referrer=utm_source%3Ddp_desktop%26utm_medium%3Ddp_browser%26utm_campaign%3Ddp_website_traffic',
 'https://itunes.apple.com/app/apple-store/id1321271821?pt=376223&ct=dp_desktop_dp_website_traffic&mt=8',
 'javascript:dpSubscription.handleAdsFreeIconClick()',
 '/?lang=hi',
 '/settings/drikpanchang-settings.html',
 "javascript:dpSettingsToolbar.handleThemeOptionClick('modern')",
 '/astronomy/sunrisemoonrise/daily/sunrisemoonrise.html',
 'javascript:dpSubscription.handleAdsFreeIconClick()',
 '/',
 '/panchang/hindu-panchangs.html',
 '/panchang/month-panchang.html',
 '/panchang/day-panchang.html',
 '/assamese/assamese-month-panjika.html',
 '/bengali/bengali-month-panjika.html',
 '/tamil/tamil-month-panchangam.html',
 '/oriya/oriya-panji.html',
 '/malayalam/malayalam-month-calendar.html',
 '/marathi/panchang/marathi-month-panchang.html',
 '/gujarati/panchang/gujarati-month-panchang.html',
 '/kannada/panchangam/kannada-

In [17]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [19]:
def get_links_user_prompt(url):
    user_prompt = f"""
    Here is the list of links on the website {url} -
    Please decide which of these are relevant web links for a brochure about the company, 
    respond with the full https URL in JSON format.
    Do not include Terms of Service, Privacy, email links.
    
    Links (some might be relative links):
    
    """
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [21]:
print(get_links_user_prompt("https://www.drikpanchang.com/"))


    Here is the list of links on the website https://www.drikpanchang.com/ -
    Please decide which of these are relevant web links for a brochure about the company, 
    respond with the full https URL in JSON format.
    Do not include Terms of Service, Privacy, email links.
    
    Links (some might be relative links):
    
    /
/
https://play.google.com/store/apps/details?id=com.drikp.core&referrer=utm_source%3Ddp_desktop%26utm_medium%3Ddp_browser%26utm_campaign%3Ddp_website_traffic
https://itunes.apple.com/app/apple-store/id1321271821?pt=376223&ct=dp_desktop_dp_website_traffic&mt=8
javascript:dpSubscription.handleAdsFreeIconClick()
/?lang=hi
/settings/drikpanchang-settings.html
javascript:dpSettingsToolbar.handleThemeOptionClick('modern')
/astronomy/sunrisemoonrise/daily/sunrisemoonrise.html
javascript:dpSubscription.handleAdsFreeIconClick()
/
/panchang/hindu-panchangs.html
/panchang/month-panchang.html
/panchang/day-panchang.html
/assamese/assamese-month-panjika.html
/bengali

In [23]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links

In [25]:
select_relevant_links("https://www.drikpanchang.com/")

{'links': [{'type': 'careers page',
   'url': 'https://www.drikpanchang.com/contactus/careers/job-openings.html'},
  {'type': 'contact page',
   'url': 'https://www.drikpanchang.com/contactus/contactus.html'},
  {'type': 'apps main page',
   'url': 'https://www.drikpanchang.com/apps/apps.html'},
  {'type': 'android apps page',
   'url': 'https://www.drikpanchang.com/apps/android/drikpanchang-apps.html'},
  {'type': 'ios apps page',
   'url': 'https://www.drikpanchang.com/apps/ios/drikpanchang-ios-apps.html'},
  {'type': 'google play store',
   'url': 'https://play.google.com/store/apps/details?id=com.drikp.core&referrer=utm_source%3Ddp_desktop%26utm_medium%3Ddp_browser%26utm_campaign%3Ddp_website_traffic'},
  {'type': 'apple App Store',
   'url': 'https://itunes.apple.com/app/apple-store/id1321271821?pt=376223&ct=dp_desktop_dp_website_traffic&mt=8'},
  {'type': 'chrome web store',
   'url': 'https://chrome.google.com/webstore/detail/drik-panchang/aohofenjhojnoploandaecgkfejilimm'},
  {

In [29]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


{'links': [{'type': 'company homepage', 'url': 'https://huggingface.co/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'Discord community', 'url': 'https://huggingface.co/join/discord'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Developer endpoints', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'Status page', 'url': 'https://status.huggingface.co/'}]}

In [32]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [34]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Lightricks/LTX-2
Updated
1 day ago
•
330k
•
688
tencent/HY-MT1.5-1.8B
Updated
9 days ago
•
8.05k
•
689
Qwen/Qwen-Image-2512
Updated
9 days ago
•
19.7k
•
549
nvidia/nemotron-speech-streaming-en-0.6b
Updated
4 days ago
•
1.07k
•
251
naver-hyperclovax/HyperCLOVAX-SEED-Think-32B
Updated
4 days ago
•
30.2k
•
328
Browse 2M+ models
Spaces
Running
Featured
4.03k
Wan2.2 Animate
👁
4.03k
Wan2.2 Animate
Running
on
Zero
1.2k
Z Image Turbo
🖼
1.2k
Generate stunning images from text descriptions in seconds
Running
on
Zero
192
Qwen Image Multiple Angl

In [35]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

In [38]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
    You are looking at a company called: {company_name}
    Here are the contents of its landing page and other relevant pages;
    use this information to build a short brochure of the company in markdown without code blocks.\n\n
    """
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [40]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links


'\n    You are looking at a company called: HuggingFace\n    Here are the contents of its landing page and other relevant pages;\n    use this information to build a short brochure of the company in markdown without code blocks.\n\n\n    ## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nLightricks/LTX-2\nUpdated\n1 day ago\n•\n330k\n•\n688\ntencent/HY-MT1.5-1.8B\nUpdated\n9 days ago\n•\n8.05k\n•\n689\nQwen/Qwen-Image-2512\nUpdated\n9 days ago\n•\n19.7k\n•\n549\nnvidia/nemotron-speech-streaming-en-0.6b\nUpdated\n4 days ago\n•\n1.07k\n•\n251\nnaver-hyperclovax/HyperCLOVAX-SEED-Think-32B\nUpdated\n4 days ago\n•\n30.2k\n•\n328\nBrowse 2M+ models\nSpaces\nRunnin

In [42]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [44]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the leading AI community and collaboration platform shaping the future of machine learning. It serves as a vibrant hub where ML engineers, researchers, and developers come together to create, discover, share, and experiment with open-source models, datasets, and applications. Trusted by a fast-growing global community, Hugging Face is dedicated to building an **open and ethical AI future**.

---

## What We Offer

- **Models:** Access and contribute to over **2 million** state-of-the-art machine learning models, spanning multiple modalities such as text, image, video, audio, and even 3D.
- **Datasets:** Browse and collaborate on more than **500,000 datasets** from diverse domains to fuel your ML projects.
- **Spaces:** Host and share interactive ML applications and demos effortlessly within the community.
- **Community:** Join a passionate and supportive network of AI enthusiasts, scientists, engineers, and developers committed to collaboration and innovation.
- **Enterprise Solutions:** Leverage our paid Compute and Enterprise offerings to accelerate your AI initiatives with scalability, performance, and dedicated support.
- **Open Source Stack:** Utilize the cutting-edge open-source tools developed by Hugging Face to move faster and build better ML systems.

---

## Why Choose Hugging Face?

- **Collaborative Platform:** Unlimited public hosting of projects ensuring accessibility and collective growth.
- **Multi-Modal Exploration:** Support for a wide range of data types including text, images, speech, video, and more.
- **Build & Showcase:** Enhance your professional portfolio by sharing your contributions with the global ML community.
- **Fast Innovation:** Regularly updated trending models and datasets keep you at the forefront of AI research and application.
- **Ethical AI Focus:** Commitment to transparency, openness, and ethical standards in AI development.

---

## Our Customers

Our platform is widely used by:

- Independent AI researchers and developers
- Academia and scientific research institutions
- Leading tech companies integrating AI solutions
- Enterprises seeking scalable, cutting-edge AI capabilities
- Open-source enthusiasts and contributors worldwide

---

## Company Culture

Hugging Face fosters an **inclusive, open, and passionate culture** built on:

- Collaboration: Encouraging knowledge-sharing and teamwork across global boundaries.
- Innovation: Constantly pushing the envelope of what's possible with open-source AI technologies.
- Ethics & Responsibility: Promoting transparency and fairness in AI advancements.
- Community-Driven Development: Listening closely to user feedback and community needs to shape product evolution.

---

## Careers at Hugging Face

Join a pioneering team committed to democratizing AI. We seek talented, motivated individuals who want to:

- Build impactful machine learning tools and infrastructure
- Contribute to open-source projects with global reach
- Work alongside diverse experts passionate about AI and community
- Advance ethical and responsible AI practices

Find openings for roles in engineering, research, product, and more on the Hugging Face careers page. Be part of the AI revolution with a company that values creativity, learning, and collaboration.

---

## Connect and Join Us

- **Explore Models & Apps:** [huggingface.co/models](https://huggingface.co/models)
- **Browse Datasets:** [huggingface.co/datasets](https://huggingface.co/datasets)
- **Try Spaces Apps:** [huggingface.co/spaces](https://huggingface.co/spaces)
- **Sign Up:** Join the community and start collaborating at [huggingface.co](https://huggingface.co)
- **Enterprise Inquiries:** Contact us for solutions tailored to your organization's AI needs.

---

**Hugging Face — The AI community building the future.**  
Collaborate. Innovate. Share. Together we build better AI.

---

*Brand Colors: #FFD21E, #FF9D00, #6B7280*

In [58]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [60]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is a pioneering AI community and collaboration platform dedicated to building the future of machine learning. Renowned as the home of machine learning, Hugging Face empowers a global community of engineers, scientists, and innovators to create, discover, and share state-of-the-art machine learning models, datasets, and applications.

---

## What We Offer

- **Extensive Model Hub:**  
  Explore and collaborate on over 2 million open-source machine learning models across text, image, video, audio, and even 3D modalities. Whether it's natural language processing, computer vision, or speech recognition, Hugging Face provides models ready to accelerate your AI projects.

- **Vast Dataset Repository:**  
  Access more than 500,000 datasets, continuously updated and curated for diverse machine learning tasks.

- **Interactive Spaces:**  
  Build, share, and run machine learning applications and demos in interactive Spaces hosted on the platform, enhancing collaboration and showcasing innovations.

- **Open Source Software Stack:**  
  Utilize the robust Hugging Face open-source ecosystem to move faster in developing and deploying ML solutions.

- **Enterprise Solutions:**  
  Benefit from scalable paid compute and enterprise-grade services designed for production and advanced projects.

---

## Community & Collaboration

Hugging Face thrives on its fast-growing, vibrant community where collaboration and sharing are at the core. The platform fosters an open and ethical AI future by providing a centralized hub for:

- Sharing models, datasets, and applications publicly to build a rich portfolio and ML profile  
- Learning from peers through resources, documentation, and continuous updates  
- Contributing to open-source AI projects and accelerating innovation together

---

## Culture & Values

- **Open and Ethical AI:**  
  Hugging Face is committed to advancing machine learning with transparency, openness, and ethical responsibility.

- **Empowerment Through Collaboration:**  
  The platform is designed to empower the next generation of AI professionals by providing access to cutting-edge tools, data, and community support.

- **Innovation at Scale:**  
  With a seamless blend of open source and enterprise offerings, Hugging Face supports innovators from hobbyists to industry leaders.

---

## Customers & Users

Hugging Face serves a diverse spectrum of users including:

- Machine learning researchers and scientists  
- AI engineers and developers building production systems  
- Enterprises seeking scalable AI solutions  
- Educators and learners in AI and data science fields  
- Open source contributors and AI enthusiasts worldwide  

Brands like NVIDIA, Tencent, Facebook research, and others actively contribute and use Hugging Face’s resources.

---

## Careers & Opportunities

Hugging Face is constantly growing and looking for passionate individuals who believe in the power of open source and AI innovation. By joining Hugging Face, you become part of a forward-thinking, inclusive, and dynamic culture committed to building the future of AI.

Explore opportunities in:

- Machine Learning Research  
- Software Engineering  
- Community Management  
- Product Development  
- Enterprise Solutions  

Join us to collaborate with a world-class team and contribute to impactful projects shaping the future of artificial intelligence.

---

## Get Started

- **Explore:** Browse over 2 million models and 500,000 datasets today  
- **Collaborate:** Host your own models, datasets, and AI apps with the community  
- **Accelerate:** Take advantage of paid compute and enterprise solutions for scalable projects  
- **Community:** Join the global AI community fueling innovation  

**Website:** [huggingface.co](https://huggingface.co)  
**Sign Up:** Create your free account and build your AI portfolio now!

---

## Brand Identity

- Signature Colors: Yellow (#FFD21E), Orange (#FF9D00), Gray (#6B7280)  
- Recognizable Logos available in .svg, .png, and .ai formats ensuring versatility across platforms

---

Hugging Face – Where machine learning meets community, innovation, and ethical AI to build the future together.

In [50]:
translation_system_prompt = """
You are a professional marketing translator.
Translate the brochure into the requested language.
Rules:
- Keep formatting (headings, bullets, line breaks).
- Keep brand/product names and URLs unchanged.
- Do NOT add new claims or information.
- Keep it natural and brochure-like, not literal.
"""

def get_translation_user_prompt(brochure_text: str, target_language: str) -> str:
    return f"""
Translate the following brochure into {target_language}.

BROCHURE:
{brochure_text}
""".strip()

In [52]:
from IPython.display import display, update_display, Markdown

def stream_llm(messages, model="gpt-4.1-mini", title=""):
    stream = openai.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )
    response = ""
    if title:
        display(Markdown(f"## {title}"))
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        delta = chunk.choices[0].delta
        response += getattr(delta, "content", "") or ""
        update_display(Markdown(response), display_id=display_handle.display_id)

    return response


In [54]:
def stream_brochure_with_translation(company_name, url, target_language="Kannada", model="gpt-4.1-mini"):
    # Step 1: English brochure
    english_messages = [
        {"role": "system", "content": brochure_system_prompt},
        {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
    ]
    english_brochure = stream_llm(
        messages=english_messages,
        model=model,
        title="Brochure (English)"
    )

    # Step 2: Translated brochure
    translation_messages = [
        {"role": "system", "content": translation_system_prompt},
        {"role": "user", "content": get_translation_user_prompt(english_brochure, target_language)}
    ]
    translated_brochure = stream_llm(
        messages=translation_messages,
        model=model,
        title=f"Brochure ({target_language})"
    )

    return english_brochure, translated_brochure


In [56]:
eng, kan = stream_brochure_with_translation(
    company_name="Hugging Face",
    url="https://huggingface.co",
    target_language="Kannada"
)

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 15 relevant links


## Brochure (English)

# Hugging Face – The AI Community Building the Future

---

## About Hugging Face

Hugging Face is a vibrant AI community and platform dedicated to building the future of machine learning by enabling collaboration on models, datasets, and applications. With over 2 million models and 500k+ datasets available, Hugging Face serves as the home for machine learning practitioners, researchers, and enterprises worldwide to create, discover, share, and advance AI technology together.

The platform supports all AI modalities including text, image, video, audio, and 3D, providing an open, collaborative environment to accelerate machine learning development.

---

## Key Features

- **Extensive Model Hub:** Browse and leverage over 2 million ML models ranging from image generation to language understanding, regularly updated and maintained by the community and partners.
- **Rich Dataset Repository:** Access and collaborate on 500,000+ datasets spanning diverse domains.
- **Spaces:** Deploy and share interactive machine learning demos and applications easily in a serverless environment.
- **Open Source Stack:** Build faster and better by utilizing Hugging Face’s open source tools and libraries widely adopted across ML ecosystems.
- **Portfolio Building:** Share your creations and build your ML profile visible to the global AI community.

---

## Enterprise Solutions

For organizations looking to scale AI initiatives with enterprise-grade capabilities, Hugging Face offers customizable Team and Enterprise plans including:

- Single Sign-On (SSO) integration and advanced security policies
- Granular access controls & audit logs for compliant governance
- Analytics dashboards to monitor usage and optimize resources
- Enhanced compute options and 5x quota boost on GPU resources
- Private dataset viewers and storage expansion
- Priority support from Hugging Face experts
- Flexible billing options (per user/month or yearly commitments)

Enterprises worldwide leverage Hugging Face’s platform to accelerate AI development while maintaining data privacy, security, and robust controls.

---

## Pricing Overview

- **Pro Account:** For individual users at $9/month, offering enhanced private storage, increased compute credits, priority queueing, and exclusive features like Spaces Dev Mode.
- **Team Plan:** Starting at $20/user/month, enables quick team collaboration with centralized management, ideal for growing organizations.
- **Enterprise Plan:** Custom pricing and options catering to larger organizations requiring advanced security, support, and flexibility.

---

## Company Culture

Hugging Face thrives on community, openness, and innovation. Their platform is a testament to collaborative growth, encouraging machine learning enthusiasts, researchers, and developers to contribute, share, and learn from one another. The company puts a strong emphasis on building the future of AI responsibly, inclusively, and transparently.

---

## Careers at Hugging Face

Hugging Face offers exciting career opportunities for passionate individuals eager to push the boundaries of AI. Whether focused on research, engineering, product, or community development, working here means joining a forward-thinking global team dedicated to nurturing the AI ecosystem through open collaboration.

Current openings span various roles — visit their careers page to explore positions and join a company that shapes the future of machine learning.

---

## Join the Future of AI with Hugging Face

Whether you're an individual creator, a research team, or a growing enterprise, Hugging Face provides the tools, infrastructure, and community support to accelerate your AI development journey.

**Explore more: [huggingface.co](https://huggingface.co)**  
Sign up to start building, sharing, and collaborating today!

---

*Hugging Face – Empowering the world to innovate in AI together.*

## Brochure (Kannada)

# Hugging Face – ಭವಿಷ್ಯವನ್ನು ನಿರ್ಮಿಸುವ AI ಸಮುದಾಯ

---

## Hugging Face ಬಗ್ಗೆ

Hugging Face ಒಂದು ಜೋಶಕಾರಿ AI ಸಮುದಾಯ ಮತ್ತು ವೇದಿಕೆ, ಇದು ಯಂತ್ರ ಕಲಿಕೆಯ ಭವಿಷ್ಯವನ್ನು ರೂಪಿಸುವತ್ತModels, ಡೇಟಾಸೆಟ್‌ಗಳು ಮತ್ತು ಅಪ್ಲಿಕೇಶನ್‌ಗಳ ಮೇಲಿನ ಸಹಕಾರವನ್ನು ಸಾಧ್ಯಮಾಡಿಸುವಲ್ಲಿ ತೊಡಗಿಸಿಕೊಂಡಿದೆ. 2 ಮಿಲಿಯನ್‌ಗೂ ಹೆಚ್ಚು ಮಾದರಿಗಳು ಮತ್ತು 500,000+ ಡೇಟಾಸೆಟ್‌ಗಳೊಂದಿಗೆ, Hugging Face ಯಂತ್ರ ಕಲಿಕೆಯ ತಜ್ಞರು, ಸಂಶೋಧಕರು ಮತ್ತು ವಿಪಣಿ ಸಂಸ್ಥೆಗಳು ಒಟ್ಟಾಗಿ AI ತಂತ್ರಜ್ಞಾನವನ್ನು ಸೃಷ್ಟಿಸಲು, ಅನ್ವೇಷಿಸಲು, ಹಂಚಿಕೊಳ್ಳಲು ಮತ್ತು ಮುನ್ನಡೆಸಲು ನಿಲುವಣಿಗೆ.

ಈ ವೇದಿಕೆ ಪಠ್ಯ, ಚಿತ್ರ, ವಿಡಿಯೋ, ಧ್ವನಿ ಮತ್ತು 3D ಸೇರಿದಂತೆ ಎಲ್ಲ AI ಮಾಧ್ಯಮಗಳನ್ನು ಬೆಂಬಲಿಸುತ್ತದೆ ಮತ್ತು ಯಂತ್ರ ಕಲಿಕೆಯ ಅಭಿವೃದ್ಧಿಯನ್ನು ವೇಗಗೊಳಿಸುವ ಒಂದು ತೆರೆಯಲ್ಪಟ್ಟ, ಸಹಕಾರಿಯ ವಾತಾವರಣವನ್ನು ಒದಗಿಸುತ್ತದೆ.

---

## ಪ್ರಮುಖ ವೈಶಿಷ್ಟ್ಯಗಳು

- **ವಿಸ್ತೃತ ಮಾದರಿ ಹಬ್:** ಚಿತ್ರ ರಚನೆದಿಂದ ಭಾಷೆ ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವವರೆಗೆ 2 ಮಿಲಿಯನ್‌ಕ್ಕೂ ಹೆಚ್ಚು ML ಮಾದರಿಗಳನ್ನು ವೀಕ್ಷಿಸಿ ಹಾಗೂ ಉಪಯೋಗಿಸಿ, ಸಮುದಾಯ ಮತ್ತು ಪಾಲುದಾರರಿಂದ ನಿಯಮಿತ ಸಲ್ಪನೆಯಲ್ಲಿ ನವೀಕರಿಸಲಾಗುತ್ತದೆ.
- **ಸಂಪನ್ನ ಡೇಟಾಸೆಟ್ ಸಂಗ್ರಹ:** ವಿವಿಧ ಕ್ಷೇತ್ರಗಳಿಗೆ ಸಂಬಂಧಿಸಿದ 500,000+ ಡೇಟಾಸೆಟ್‌ಗಳನ್ನು ಒದಗಿಸುತ್ತದೆ ಹಾಗೂ ಸಹಕಾರಕ್ಕೆ ಅವಕಾಶ.
- **ಸ್ಪೇಸಸ್:** ಸರ್ವರ್‌ಲೆಸ್ ಪರಿಸರದಲ್ಲಿ ಸುಲಭವಾಗಿ ಸಂವಾದಾತ್ಮಕ ಯಂತ್ರ ಕಲಿಕೆ ಡೆಮೋಗಳು ಮತ್ತು ಅಪ್ಲಿಕೇಶನ್‌ಗಳನ್ನು ನಿಯೋಜಿಸಿ ಹಂಚಿಕೊಳ್ಳಿ.
- **ತೆರೆದ ಮೂಲ ಸ್ಟ್ಯಾಕ್:** ವೈಶ್ವಿಕ ML ಪರಿಸರಗಳಲ್ಲಿ ವ್ಯಾಪಕವಾಗಿ ಬಳಸಲಾಗುವ Hugging Face ತೆರೆದ ಮೂಲ ಸಾಧನಗಳು ಮತ್ತು ಲೈಬ್ರರಿಗಳನ್ನು ಉಪಯೋಗಿಸಿ ವೇಗವಾಗಿ ಮತ್ತು ಉತ್ತಮವಾಗಿ ನಿರ್ಮಿಸಿ.
- **ಪೋರ್ಟ್ಫೋಲಿಯೊ ನಿರ್ಮಾಣ:** ನಿಮ್ಮ ಸೃಷ್ಟಿಗಳನ್ನು ಹಂಚಿಕೊಳ್ಳಿ ಮತ್ತು ನಿಮ್ಮ ML ಪ್ರೊಫೈಲ್ ಅನ್ನು ಜಾಗತಿಕ AI ಸಮುದಾಯದಲ್ಲಿ ಗೋಚರಿಸಿರಿ.

---

## ಸಂಸ್ಥೆ ಪರಿಹಾರಗಳು

AI ಯೋಜನೆಗಳನ್ನು ಸಂಸ್ಥೆಯಲ್ಲಿ ವಿಸ್ತರಿಸಲು ಬಯಸುವ ಸಂಸ್ಥೆಗಳಿಗಾಗಿ, Hugging Face ತನ್ನ ಬದಲಾಯಿಸಬಹುದಾದ ಟೀಮ್ ಮತ್ತು ಎಂಟರ್‌ಪ್ರೈಸ್ ಯೋಜನೆಗಳನ್ನು ನೀಡುತ್ತದೆ, ಅವುಗಳಲ್ಲಿ:

- ಸಿಂಗಲ್ ಸೈನ್-ಆನ್ (SSO) ಸಂಯೋಜನೆ ಮತ್ತು ಅಗ್ರ ಕ್ರಮರಕ್ಷಣಾ ನೀತಿಗಳು  
- ನಿಯಂತ್ರಿತ ಪ್ರವೇಶ ನಿಯಂತ್ರಣಗಳು ಮತ್ತು ಅನುಸರಣೆಗಾಗಿ ಆಡಿಟ್ ಲಾಗ್ಗಳು  
- ಉಪಯೋಗವನ್ನು ಗಮನಿಸಿ ಮತ್ತು ಸಂಪನ್ಮೂಲಗಳನ್ನು ಗರಿಷ್ಠಪಡಿಸಲು ವಿಶ್ಲೇಷಣಾ ಡ್ಯಾಶ್‌ಬೋರ್ಡ್‌ಗಳು  
- ಸುಧಾರಿತ ಗಣನೆ ಆಯ್ಕೆಗಳು ಮತ್ತು GPU ಸಂಪನ್ಮೂಲಗಳ ಮೇಲೆ 5 ಮಡಿಕೆ ಕ್ವೋಟಾ ಬೆಳವಣಿಗೆ  
- ಖಾಸಗಿ ಡೇಟಾಸೆಟ್ ವೀಕ್ಷಕರು ಮತ್ತು ಸಂಗ್ರಹಣಾ ವಿಸ್ತರಣೆ  
- Hugging Face ತಜ್ಞರಿಂದ ಆದ್ಯತಾ ಬೆಂಬಲ  
- ಮುಕ್ತಾಯವಾದ ಬಿಲ್ಲಿಂಗ್ ಆಯ್ಕೆಗಳು (ಬಳಕೆದಾರನ ಪ್ರತಿ/ತಿಂಗಳು ಅಥವಾ ವರ್ಷದ ಬದ್ಧತೆಗಳು)

ವಿಶ್ವವ್ಯಾಪಿ ಸಂಸ್ಥೆಗಳು ಡೇಟಾ ಗೌಪ್ಯತೆಯು, ಭದ್ರತೆ ಮತ್ತು ಮಿರ್ದನ ನಿಯಂತ್ರಣಗಳನ್ನು ಕಾಪಾಡಿಕೊಂಡು AI ಅಭಿವೃದ್ಧಿಯನ್ನು ವೇಗಗೊಳಿಸಲು Hugging Face ವೇದಿಕೆಯನ್ನು ಬಳಸುತ್ತಾರೆ.

---

## ಬೆಲೆ ಮಾಹಿತಿ

- **ಪ್ರೊ ಖಾತೆ:** ವ್ಯಕ್ತಿಗತ ಬಳಕೆದಾರರಿಗೆ $9/ತಿಂಗಳು, ಹೆಚ್ಚಾದ ಖಾಸಗಿ ಸಂಗ್ರಹಣೆ, ಹೆಚ್ಚಾದ ಗಣನೆ ಕ್ರೆಡಿಟ್, ಆದ್ಯತಾ ಕ್ಯೂ ಮತ್ತು ವಿಶೇಷ ವೈಶಿಷ್ಟ್ಯಗಳು (Spaces ಡೆವ್ ಮೋಡ್) ಒದಗಿಸುವದು.  
- **ಟೀಮ್ ಯೋಜನೆ:** ಪ್ರತಿ ಬಳಕೆದಾರರು/ತಿಂಗಳು $20ರಿಂದ ಪ್ರಾರಂಭ, ತಂಡದ ಒಕ್ಕೂಟವನ್ನು ಮತ್ತು ಕೇಂದ್ರೀಕೃತ ನಿರ್ವಹಣೆಯನ್ನು ಸುಲಭಗೊಳಿಸಲು, ಬೆಳೆದ ಅರ್ಜಿಕೆಗಳಿಗೆ ತುಂಬ ಸೂಕ್ತ.  
- **ಎಂಟರ್‌ಪ್ರೈಸ್ ಯೋಜನೆ:** ಹೆಚ್ಚಿನ ಭದ್ರತೆ, ಬೆಂಬಲ ಮತ್ತು ಲವಚಿಕತೆಗೆ ಅಗತ್ಯವಿರುವ ದೊಡ್ಡ ಸಂಸ್ಥೆಗಳಿಗಾಗಿ ಕಸ್ಟಮ್ ಬೆಲೆ ಮತ್ತು ಆಯ್ಕೆಗಳು.

---

## ಕಂಪನಿ ಸಂಸ್ಕೃತಿ

Hugging Face ಸಮುದಾಯ, ತೆರೆಯುಹೊಗುವಿಕೆ ಮತ್ತು ನವೀನತೆಯನ್ನು ಹಿರಿಯವಾಗಿ ಅಂಗೀಕರಿಸಿದೆ. ಅವರ ವೇದಿಕೆ ಸಹಕಾರದ ಗಣನೀಯ ಬೆಳವಣಿಗೆಯ ಸಾಕ್ಷಿ, ಯಂತ್ರ ಕಲಿಕೆಯ ರಂಜಕರು, ಸಂಶೋಧಕರು ಮತ್ತು ಅಭಿವೃದ್ಧಿಪಡಿಸುವವರನ್ನು ಸೇರಿಸಿ, ಹಂಚಿಕೊಳ್ಳಲು ಮತ್ತು ಪರಸ್ಪರದಿಂದ ಕಲಿಯಲು ಉತ್ತೇಜನ ನೀಡುತ್ತದೆ. ಕಂಪನಿ ಜವಾಬ್ದಾರಿಯಿಂದ, ಸಮಗ್ರವಾಗಿ ಮತ್ತು ಪಾರದರ್ಶಕವಾಗಿ AI ಭವಿಷ್ಯವನ್ನು ನಿರ್ಮಿಸಲು ವಿಶೇಷ ಮಹತ್ವ ನೀಡುತ್ತದೆ.

---

## Hugging Face ನಲ್ಲಿ ಉದ್ಯೋಗ ಅವಕಾಶಗಳು

AI ನಡಿಯನ್ನು ಮುಂದುವರಿಸಲು ಉತ್ಸಾಹಿ ವ್ಯಕ್ತಿಗಳಿಗೆ Hugging Face ಆಕರ್ಷಕ ಉದ್ಯೋಗ ಅವಕಾಶಗಳನ್ನು ನೀಡುತ್ತದೆ. ಸಂಶೋಧನೆ, ಎಂಜಿನಿಯರಿಂಗ್, ಉತ್ಪನ್ನ ಅಥವಾ ಸಮುದಾಯ ಅಭಿವೃದ್ಧಿ ಕ್ಷೇತ್ರಗಳಾದರೂ, ಇಲ್ಲಿ ಕೆಲಸ ಮಾಡುವುದು ತೆರೆಯುಹೊಗುವ ಐಡೀಯಾಗಳನ್ನು ಹುರಿದುಂಬಿಸುವ ಜಾಗತಿಕ ತಂಡದ ಭಾಗವಾಗುವುದು.

ಪ್ರಸ್ತುತ ವಿವಿಧ ಹುದ್ದೆಗಳ ಬಗ್ಗೆ ತಿಳಿಯಲು ಅವರ ಉದ್ಯೋಗ ಪುಟವನ್ನು ಭೇಟಿ ಮಾಡಿ ಮತ್ತು ಯಂತ್ರ ಕಲಿಕೆಯ ಭವಿಷ್ಯವನ್ನು ರೂಪಿಸುವ ಸಂಸ್ಥೆಗೆ ಸೇರಿಕೊಳ್ಳಿ.

---

## Hugging Face ಜೊತೆಗೆ AI ಭವಿಷ್ಯವನ್ನು ಸೇರಿ

ನೀವು ವೈಯಕ್ತಿಕ ಸೃಷ್ಟಿಕರ್ತೆ, ಸಂಶೋಧನಾ ತಂಡ ಅಥವಾ ಬೆಳೆಯುತ್ತಿರುವ ಸಂಸ್ಥೆಯಾಗಿದ್ದರೂ, Hugging Face ನಿಮ್ಮ AI ಅಭಿವೃದ್ಧಿ ಪ್ರಯಾಣವನ್ನು ವೇಗಗೊಳಿಸಲು ಉಪಕರಣಗಳು, ಮೂಲಸೌಕರ್ಯ ಮತ್ತು ಸಮುದಾಯ ಬೆಂಬಲವನ್ನು ಒದಗಿಸುತ್ತದೆ.

**ಹೆಚ್ಚು ತಿಳಿದುಕೊಳ್ಳಿ: [huggingface.co](https://huggingface.co)**  
ಇಂದೆಯೇ ಸೈನ್ ಅಪ್ ಮಾಡಿ ನಿರ್ಮಿಸಲು, ಹಂಚಿಕೊಳ್ಳಲು ಮತ್ತು ಸಹಕರಿಸಲು ಆರಂಭಿಸಿ!

---

*Hugging Face – ಜಗತ್ತಿನವರು ಸೇರಿ AI ಯಲ್ಲಿ ನವೀನತೆಯನ್ನು ಸಾಧ್ಯಮಾಡಿಸುವಂತೆ ಪ್ರೇರೇಪಿಸುವುದು.*